In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd


worldcup_teams =['Egypt', 'Russia', 'Saudi Arabia', 'Uruguay', 'Iran', 'Morocco',
       'Portugal', 'Spain', 'Australia', 'Denmark', 'France', 'Peru',
       'Argentina', 'Croatia', 'Iceland', 'Nigeria', 'Brazil',
       'Costa Rica', 'Serbia', 'Switzerland', 'Germany', 'Mexico',
       'South Korea', 'Sweden', 'Belgium', 'United Kingdom', 'Panama', 'Tunisia',
       'Colombia', 'Japan', 'Poland', 'Senegal']


In [92]:
import unicodedata

def scrape_gdp_pp():
    url = "https://en.wikipedia.org/wiki/List_of_countries_by_GDP_(PPP)_per_capita"
    my_page = requests.get(url)
    soup = BeautifulSoup(my_page.content, "lxml")
    tables = soup.find_all('table',{'class':'wikitable sortable'})
    country_rows = tables[0].findAll('tr')
    countries_data = []

    for country in country_rows[1:]:    
        country_data = country.findAll('td')
        name = country_data[1].text
        country_name = unicodedata.normalize("NFKD", name)
        country_name = country_name.strip()
        gdp_pp = int(country_data[2].text.strip().replace(',',''))
        if (country_name in worldcup_teams):
            countries_data.append({"name":country_name,'gdp_pp':gdp_pp})

    return pd.DataFrame(countries_data)

In [93]:
gdp_df = scrape_gdp_pp()
gdp_df.head()

,gdp_pp,name
0,61360,Switzerland
1,55263,Saudi Arabia
2,52150,Iceland
3,51264,Sweden
4,50206,Germany


In [119]:
def scrape_temp():
    url = "https://en.wikipedia.org/wiki/List_of_countries_by_average_yearly_temperature"
    my_page = requests.get(url)
    soup = BeautifulSoup(my_page.content, "lxml")
    tables = soup.find_all('table',{'class':'wikitable sortable'})
    country_rows = tables[0].findAll('tr')
    countries_temp = []
    
    for country in country_rows[1:]:    
        country_data = country.findAll('td')
        name = country_data[0].text
        country_name = unicodedata.normalize("NFKD", name)
        country_name = country_name.strip()
        temp_str = country_data[1].text.strip()
        temp_str = temp_str.replace('−','-')
        temp = float(temp_str)  
        if (country_name in worldcup_teams):
            countries_temp.append({"name":country_name,'temp':temp})
    return pd.DataFrame(countries_temp)


In [120]:
temps = scrape_temp()
temps.head()

,name,temp
0,Senegal,27.85
1,Nigeria,26.80
2,Panama,25.40
3,Brazil,24.95
4,Costa Rica,24.80


In [121]:
gdp_df.dtypes


gdp_pp     int64
name      object
dtype: object

In [122]:
temps.dtypes

name     object
temp    float64
dtype: object

In [123]:
combined_df = temps.merge(gdp_df,on='name')

In [125]:
combined_df

,name,temp,gdp_pp
0,Senegal,27.85,2678
1,Nigeria,26.80,5927
2,Panama,25.40,24262
3,Brazil,24.95,15500
4,Costa Rica,24.80,17149
5,Saudi Arabia,24.65,55263
6,Colombia,24.50,14455
7,Egypt,22.10,12994
8,Australia,21.65,49882
9,Mexico,21.00,19480
